In [1]:
%load_ext autoreload

%autoreload 2
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.svm import SVC
from sklearn.metrics import roc_auc_score, roc_curve

from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import PCA
from sklearn import metrics

from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import LSTM


from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence


import warnings
import numpy as np
from collections import OrderedDict

import os

from lob_data_utils import lob, db_result, gdf_pca, model
from lob_data_utils.svm_calculation import lob_svm


sns.set_style('whitegrid')
warnings.filterwarnings('ignore')

/home/vevurka/.conda/envs/py365/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
data_length = 24000
stock = '4851'
gdf_filename_pattern = 'gdf_{}_r{}_s{}_K50'
gdf_parameters = [(0.1, 0.1), (0.01, 0.1), (0.1, 0.5), (0.01, 0.5)]

In [3]:
df_log = pd.read_csv('res_log_que.csv')   
df_log = df_log[df_log['stock'] == int(stock)]
columns = [c for c in df_log.columns if 'matthews' in c or 'roc_auc' in c]
df_log[columns]

,matthews,roc_auc,test_matthews,test_roc_auc,train_matthews,train_roc_auc,train_val_matthews,train_val_roc_auc
81,0.141509,0.602119,0.13868,0.600747,0.146122,0.601755,0.1511,0.603678


In [4]:
gdf_dfs = []
for r, s in gdf_parameters:
    gdf_dfs.append(gdf_pca.SvmGdfResults(
        stock,  r=r, s=s, data_length=data_length, data_dir='../gaussian_filter/data_gdf_not_synced',
        gdf_filename_pattern=gdf_filename_pattern))

In [5]:
gdf_dfs[0].df.columns

Index(['Unnamed: 0', 'datetime', 'gdf_0', 'gdf_1', 'gdf_10', 'gdf_11',
       'gdf_12', 'gdf_13', 'gdf_14', 'gdf_15', 'gdf_16', 'gdf_17', 'gdf_18',
       'gdf_19', 'gdf_2', 'gdf_20', 'gdf_21', 'gdf_22', 'gdf_23', 'gdf_24',
       'gdf_25', 'gdf_26', 'gdf_27', 'gdf_28', 'gdf_29', 'gdf_3', 'gdf_30',
       'gdf_31', 'gdf_32', 'gdf_33', 'gdf_34', 'gdf_35', 'gdf_36', 'gdf_37',
       'gdf_38', 'gdf_39', 'gdf_4', 'gdf_40', 'gdf_41', 'gdf_42', 'gdf_43',
       'gdf_44', 'gdf_45', 'gdf_46', 'gdf_47', 'gdf_48', 'gdf_49', 'gdf_5',
       'gdf_6', 'gdf_7', 'gdf_8', 'gdf_9', 'mid_price', 'mid_price_indicator',
       'queue_imbalance', 'bid_price', 'ask_price', 'prev_queue_imbalance'],
      dtype='object')

In [10]:
def as_keras_metric(method):
    import functools
    from keras import backend as K
    import tensorflow as tf
    @functools.wraps(method)
    def wrapper(self, args, **kwargs):
        """ Wrapper for turning tensorflow metrics into keras metrics """
        value, update_op = method(self, args, **kwargs)
        K.get_session().run(tf.local_variables_initializer())
        with tf.control_dependencies([update_op]):
            value = tf.identity(value)
        return value
    return wrapper
import tensorflow as tf
auc_roc = as_keras_metric(tf.metrics.auc)

In [7]:
y_train = gdf_dfs[0].df['mid_price_indicator'].values
from sklearn.utils import class_weight
a = np.unique(y_train)
print(a)
class_weight.compute_class_weight('balanced', a, y_train)

[0. 1.]


array([0.58766452, 3.35178073])

In [24]:
model = Sequential()
model.add(LSTM(6))
model.add(Dense(6, activation='tanh'))
model.add(Dense(6, activation='tanh'))
model.add(Dense(6, activation='tanh'))
model.add(Dense(6, activation='tanh'))
model.add(Dense(6, activation='tanh'))
model.add(Dense(6, activation='tanh'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=[auc_roc])

In [25]:
epochs = 3
batch_size = 50
score = gdf_dfs[0].train_lstm(
    model, feature_name='pca_n_gdf_que_prev', plot_name='here.png',
    fit_kwargs={'epochs': epochs, 'batch_size': batch_size, 'verbose': 1, 'shuffle': False}, 
    compile_kwargs= { 'loss': 'binary_crossentropy', 'optimizer': 'adam', 'metrics': [auc_roc]},
    class_weight={0: 0.58766452, 1: 3.35178073})

Epoch 1/3
8725/8725 [==============================] - 4s 496us/step - loss: 0.7095 - auc: 0.5257
Epoch 2/3
8725/8725 [==============================] - 0s 56us/step - loss: 0.6959 - auc: 0.5827
Epoch 3/3
8725/8725 [==============================] - 0s 55us/step - loss: 0.6895 - auc: 0.6041
Epoch 1/3
8725/8725 [==============================] - 5s 530us/step - loss: 0.6859 - auc: 0.6128
Epoch 2/3
8725/8725 [==============================] - 0s 55us/step - loss: 0.6854 - auc: 0.6392
Epoch 3/3
8725/8725 [==============================] - 0s 55us/step - loss: 0.6852 - auc: 0.6417
Epoch 1/3
8725/8725 [==============================] - 5s 568us/step - loss: 0.6635 - auc: 0.6489
Epoch 2/3
8725/8725 [==============================] - 0s 56us/step - loss: 0.6631 - auc: 0.6601
Epoch 3/3
8725/8725 [==============================] - 0s 56us/step - loss: 0.6627 - auc: 0.6613
Epoch 1/3
8725/8725 [==============================] - 5s 611us/step - loss: 0.6570 - auc: 0.6523
Epoch 2/3
8725/8725 [=====

In [27]:
print(score['matthews'], score['train_matthews'], score['test_matthews'])
score['roc_auc'], score['train_roc_auc'], score['test_roc_auc']

0.15462240845231612 0.1638715652092099 0.16414313401061825


(0.6068381147607853, 0.6132439277658932, 0.6187873460902539)

In [ ]:
## True LSTM 

In [28]:
model = Sequential()
model.add(LSTM(12, input_shape=(10, 4)))
model.add(Dense(12, activation='tanh'))
model.add(Dense(6, activation='tanh'))
model.add(Dense(4, activation='tanh'))
model.add(Dense(2, activation='tanh'))
# model.add(Dense(6, activation='tanh'))
# model.add(Dense(6, activation='tanh'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=[auc_roc])

In [29]:
epochs = 3
batch_size = 50
score = gdf_dfs[0].train_lstm(
    model, feature_name='pca_n_gdf_que_prev', plot_name='here.png', n_steps=10,
    fit_kwargs={'epochs': epochs, 'batch_size': batch_size, 'verbose': 1, 'shuffle': False}, 
    compile_kwargs= { 'loss': 'binary_crossentropy', 'optimizer': 'adam', 'metrics': [auc_roc]},
    class_weight={0: 0.58766452, 1: 3.35178073})

Epoch 1/3
8720/8720 [==============================] - 7s 848us/step - loss: 0.7034 - auc: 0.5653
Epoch 2/3
8720/8720 [==============================] - 1s 123us/step - loss: 0.6903 - auc: 0.6040
Epoch 3/3
8720/8720 [==============================] - 1s 123us/step - loss: 0.6876 - auc: 0.6186
Epoch 1/3
8720/8720 [==============================] - 8s 877us/step - loss: 0.6841 - auc: 0.6157
Epoch 2/3
8720/8720 [==============================] - 1s 119us/step - loss: 0.6834 - auc: 0.6418
Epoch 3/3
8720/8720 [==============================] - 1s 120us/step - loss: 0.6829 - auc: 0.6445
Epoch 1/3
8720/8720 [==============================] - 8s 916us/step - loss: 0.6611 - auc: 0.6504
Epoch 2/3
8720/8720 [==============================] - 1s 121us/step - loss: 0.6603 - auc: 0.6641
Epoch 3/3
8720/8720 [==============================] - 1s 122us/step - loss: 0.6597 - auc: 0.6657
Epoch 1/3
8720/8720 [==============================] - 8s 942us/step - loss: 0.6540 - auc: 0.6601
Epoch 2/3
8720/8720 

In [30]:
print(score['matthews'], score['train_matthews'], score['test_matthews'])
score['roc_auc'], score['train_roc_auc'], score['test_roc_auc']

0.15613824278721528 0.17062989490760755 0.18318639236055428


(0.608856615583336, 0.6173021372460791, 0.6313930110995783)